In [4]:
import pandas as pd
import numpy as np
import ta
import time
from binance import Client
from binancekeys import api_key, secret_key

In [6]:
client = Client(api_key, secret_key)

In [7]:
def getminutedata(symbol, interval, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol,
                                                     interval,
                                                     lookback + ' min ago UTC'))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    
    return frame

In [11]:
df = getminutedata('ADAUSDT', '1m', '100')

In [14]:
def applytechnicals(df):
    df['%K'] = ta.momentum.stoch(df.High, df.Low, df.Close, window=14, smooth_window=3)
    df['%D'] = df['%K'].rolling(3).mean()
    df['rsi'] = ta.momentum.rsi(df.Close, window=14)
    df['macd'] = ta.trend.macd_diff(df.Close)
    df.dropna(inplace=True)

In [15]:
applytechnicals(df)

In [18]:
class Signals:
    def __init__(self, df, lags):
        self.df = df
        self.lags = lags
        
    def gettrigger(self):
        dfx = pd.DataFrame()
        for i in range(self.lags + 1):
            mask = (self.df['%K'].shift(i) < 20) & (self.df['%D'].shift(i) < 20)
            dfx = dfx.append(mask, ignore_index=True)
            
        return dfx.sum(axis=0)
    
    def decide(self):
        self.df['trigger'] = np.where(self.gettrigger(), 1, 0)
        self.df['Buy'] = np.where((self.df.trigger) &
                                 (self.df['%K'].between(20, 80)) & (self.df['%D'].between(20, 80)) &
                                 (self.df.rsi > 50) & (self.df.macd > 0), 1, 0)
        

In [23]:
inst = Signals(df, 25)

In [24]:
inst.decide()

C:\Users\bgour\AppData\Local\Temp\ipykernel_11860\2673246236.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx = dfx.append(mask, ignore_index=True)
C:\Users\bgour\AppData\Local\Temp\ipykernel_11860\2673246236.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx = dfx.append(mask, ignore_index=True)
C:\Users\bgour\AppData\Local\Temp\ipykernel_11860\2673246236.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx = dfx.append(mask, ignore_index=True)
C:\Users\bgour\AppData\Local\Temp\ipykernel_11860\2673246236.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfx = dfx.append(mask, ignore_index=True)
C:\Users\bgour\AppDa

In [26]:
def strategy(pair, qty, open_position=False):
    df = getminutedata(pair, '1m', '100')
    applytechnicals(df)
    inst = Signals(df, 25)
    inst.decide()
    print(f'current Close is '+str(df.Close.iloc[-1]))
    if df.Buy.iloc[-1]:
        order = client.create_order(symbol=pair,
                                   side='BUY',
                                   type='MARKET',
                                   quantity=qty)
        print(order)
        print()
        buyprice = float(order['fills'][0]['price'])
        open_position=True
        
    while open_position:
        time.sleep(0.5)
        df = getminutedata(pair, '1m', '2')
        print(f'current Close' + str(df.Close.iloc[-1]))
        print(f'current Target' + str(buyprice * 1.005))
        print(f'current Stop is' + str(buyprice * 0.995))
        
        if df.Close[-1] <= buyprice * 0.995 or df.Close[-1] >= buyprice * 1.005:
            order = client.create_order(symbol=pair,
                                       side='SELL',
                                       type='MARKET',
                                       quantity=qty)
            print(order)
            print()
            break
            

In [ ]:
'''
while True:
    strategy('ADAUSDT', 50)
    time.sleep(0.5)
'''